In [120]:
import pandas as pd
import numpy as np

## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки [data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). 

Для таблицы transactions используйте только первые n=1000000 строк. При чтении файлов обратите внимание на разделители внутри каждого из файлов - они могут различаться!

In [121]:
tr_mcc_codes = pd.read_csv('../Data/tr_mcc_codes.csv', delimiter=';')
tr_types = pd.read_csv('../Data/tr_types.csv', delimiter=';')
transactions = pd.read_csv('../Data/transactions.csv', delimiter=',', nrows=1_000_000)
gender_train = pd.read_csv('../Data/gender_train.csv', delimiter=',')

###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;


Задания 1-4 делаются без использования merge!

## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'


In [122]:
sample = transactions['tr_type'].sample(1000)
sample = pd.merge(sample, tr_types)
len(sample[sample['tr_description'].str.contains('POS|ATM')])

373

## Задание 2 


1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости (вывести для них tr_description тоже).

In [123]:
tr_type_freq = transactions['tr_type'].value_counts(sort=True)
tr_type_freq = tr_type_freq[:10]

tr_type_freq = tr_type_freq.rename('freq').to_frame().reset_index(names='tr_type')
tr_type_freq = tr_type_freq.merge(tr_types[['tr_type', 'tr_description']])
tr_type_freq

,tr_type,freq,tr_description
0,1010,231117,Покупка. POS ТУ СБ РФ
1,2010,151166,Выдача наличных в АТМ Сбербанк России
2,7070,149006,Перевод на карту (с карты) через Мобильный бан...
3,1110,137658,Покупка. POS ТУ Россия
4,1030,118975,Оплата услуги. Банкоматы СБ РФ
5,2370,49830,Списание с карты на карту по операции <перевод...
6,7010,28972,Взнос наличных через АТМ (в своем тер.банке)
7,7030,26078,Перевод на карту (с карты) через АТМ (в предел...
8,7071,15781,Перевод на карту (с карты) через Мобильный бан...
9,1100,15476,Покупка. ТУ Россия


## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов

In [124]:
top_ups = transactions[transactions['amount'] > 0][['customer_id', 'amount']]
withdrawals = transactions[transactions['amount'] < 0][['customer_id', 'amount']]

top_up_groups = top_ups.groupby('customer_id')
withdraw_groups = withdrawals.groupby('customer_id')

max_total_top_ups_customer_id = top_up_groups.sum().idxmax()[0]
max_total_withdrawals_customer_id = withdraw_groups.sum().idxmax()[0]
print(f'1: {max_total_top_ups_customer_id}')
print(f'2: {max_total_withdrawals_customer_id}')

f = lambda customer_id: abs(withdraw_groups.get_group(customer_id).sum()[0] - top_up_groups.get_group(customer_id).sum()[0])
print(f'3: customer {max_total_top_ups_customer_id}: {f(max_total_top_ups_customer_id)}')
print(f'3: customer {max_total_withdrawals_customer_id}: {f(max_total_withdrawals_customer_id)}')

1: 70780820
2: 80185881
3: customer 70780820: 6216467078140.0
3: customer 80185881: 882044691.0


## Задание 4
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций для клиентов из задания 3

In [125]:
df = transactions.loc[transactions['tr_type'].isin(tr_type_freq['tr_type'])]
tr_type_amounts = df.groupby('tr_type')['amount']
print("1:\n")
print(f"mean\n {tr_type_amounts.mean()}\n")
print(f"median\n {tr_type_amounts.median()}\n")

df = transactions.loc[transactions['customer_id'] == max_total_top_ups_customer_id]
tr_type_amounts = df.groupby('tr_type')['amount']
print(f"2: customer {max_total_top_ups_customer_id}\n")
print(f"mean\n {tr_type_amounts.mean()}\n")
print(f"median\n {tr_type_amounts.median()}\n")

df = transactions.loc[transactions['customer_id'] == max_total_withdrawals_customer_id]
tr_type_amounts = df.groupby('tr_type')['amount']
print(f"2: customer {max_total_withdrawals_customer_id}\n")
print(f"mean\n {tr_type_amounts.mean()}\n")
print(f"median\n {tr_type_amounts.median()}")

1:

mean
 tr_type
1010    -19784.748640
1030     -5320.980222
1100    -44061.827262
1110    -32119.330371
2010   -136077.629325
2370   -205418.249032
7010    276391.789596
7030     86104.332909
7070     65569.831700
7071     66806.826623
Name: amount, dtype: float64

median
 tr_type
1010     -7411.52
1030     -2245.92
1100    -10188.26
1110    -11207.57
2010    -44918.32
2370    -44918.32
7010    112295.79
7030     13951.52
7070     11319.42
7071      3593.47
Name: amount, dtype: float64

2: customer 70780820

mean
 tr_type
2010   -2.941056e+06
2330   -2.382398e+06
2370   -2.218505e+06
6110    1.756293e+04
7020    1.465129e+04
7021    2.850067e+04
7030    1.406196e+04
7031    1.921036e+04
7034    1.430648e+04
7040    2.153833e+04
7070    1.384821e+04
7071    1.860057e+04
7074    2.281159e+04
Name: amount, dtype: float64

median
 tr_type
2010   -3368873.660
2330   -2245915.770
2370   -2245915.770
6110      10028.010
7020      10319.985
7021      28500.670
7030       8848.910
7031      1

## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [126]:
transactions = transactions.\
    merge(gender_train, how='left').\
    merge(tr_mcc_codes).\
    merge(tr_types)
transactions.shape

(999584, 9)

In [127]:
transactions

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,mcc_description,tr_description
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
1,39026145,6 07:08:31,4814,1030,-5614.79,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
2,39026145,8 07:06:10,4814,1030,-1122.96,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
3,39026145,11 08:49:03,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
4,39026145,11 14:12:08,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
...,...,...,...,...,...,...,...,...,...
999579,36837532,18 23:38:60,3000,6000,1297824.89,NaN,NaN,"Авиалинии, авиакомпании",Возврат покупки. ТУ СБ РФ
999580,6808459,206 19:00:10,3000,6000,124693.24,NaN,NaN,"Авиалинии, авиакомпании",Возврат покупки. ТУ СБ РФ
999581,83759123,90 22:40:57,3000,6000,218976.79,NaN,NaN,"Авиалинии, авиакомпании",Возврат покупки. ТУ СБ РФ
999582,89619970,104 23:39:35,3000,6000,408307.49,NaN,NaN,"Авиалинии, авиакомпании",Возврат покупки. ТУ СБ РФ


## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том, 
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

In [128]:
mean_male_withdrawal = transactions.loc[(transactions['amount'] < 0) & (transactions['gender'] == 1), 'amount'].mean()
mean_female_withdrawal = transactions.loc[(transactions['amount'] < 0) & (transactions['gender'] == 0), 'amount'].mean()
print(f'1: {abs(mean_male_withdrawal - mean_female_withdrawal)}')

mean_male_topup = transactions.loc[(transactions['amount'] > 0) & (transactions['gender'] == 1), 'amount'].mean()
mean_female_topup = transactions.loc[(transactions['amount'] > 0) & (transactions['gender'] == 0), 'amount'].mean()
print(f'2: {abs(mean_male_topup - mean_female_topup)}')

1: 32718.05492022467
2: 63366.57104801465


## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 типов транзакций для мужчин и для женщин, наименьших среди всех типов транзакций по полученным значениям "max_income". 
2. Выделите среди них те типы транзакций, которые встречаются одновременно и у мужчин, и у женщин

In [129]:
df = transactions
max_male_income_by_tr_type = df.loc[(df['amount'] > 0) & (df['gender'] == 1), ['tr_type', 'amount']].groupby('tr_type').max().rename(columns={'amount': 'max_income'})
max_female_income_by_tr_type = df.loc[(df['amount'] > 0) & (df['gender'] == 0), ['tr_type', 'amount']].groupby('tr_type').max().rename(columns={'amount': 'max_income'})
x = max_male_income_by_tr_type.sort_values(by='max_income')[:10]
y = max_female_income_by_tr_type.sort_values(by='max_income')[:10]
print(f"1: male:\n{x}\n")
print(f"1: female:\n{y}\n")
print(f"2: {set(x.index) & set(y.index)}")

1: male:
         max_income
tr_type            
4110        2245.92
2020        2245.92
1010       28803.87
8100       64682.37
6000       79280.83
7041       87590.72
2010       89836.63
2110      112295.79
2370      113194.15
7015      121279.45

1: female:
         max_income
tr_type            
4051        1122.96
4110        2245.92
4210        2245.92
2370        2335.75
4100        5041.86
7075        6737.75
2110       22459.16
1010       24839.83
2010       44918.32
2210       68315.82

2: {2370, 4110, 1010, 2010, 2110}


## Задание 7

1. Найдите суммы затрат по каждой категории (mcc) для мужчин и для женщин. 
2. Найдите топ 10 категорий с самыми большими относительными модулями разности в тратах для разных полов (в ответе должны присутствовать описания mcc кодов).

In [130]:
df = transactions
male_spendings_sum_by_mcc_type = df.loc[(df['amount'] < 0) & (df['gender'] == 1), ['mcc_code', 'amount']].groupby('mcc_code').sum().rename(columns={'amount': 'spendings_sum'})
female_spendings_sum_by_mcc_type = df.loc[(df['amount'] < 0) & (df['gender'] == 0), ['mcc_code', 'amount']].groupby('mcc_code').sum().rename(columns={'amount': 'spendings_sum'})
print(f"1: male:\n{male_spendings_sum_by_mcc_type}\n")
print(f"1: female:\n{female_spendings_sum_by_mcc_type}\n")

dif = (male_spendings_sum_by_mcc_type - female_spendings_sum_by_mcc_type).dropna().abs().rename(columns={'spendings_sum': 'spendings_sum_abs_dif'})
print(f"2:\n{dif.sort_values(by='spendings_sum_abs_dif', ascending=False)[:10]}")

1: male:
          spendings_sum
mcc_code               
742          -132277.93
1711         -206264.90
1731         -121542.67
1799        -1129257.68
2741         -221290.08
...                 ...
8999       -28643346.63
9222         -988205.17
9311         -373271.22
9399        -1810820.67
9402           -4581.67

[175 rows x 1 columns]

1: female:
          spendings_sum
mcc_code               
742          -386207.68
1711         -674897.69
1799         -261626.73
2741          -20746.42
3000       -74431645.27
...                 ...
8699         -521107.48
8999       -19432057.92
9222         -361143.24
9311         -406197.45
9399        -1481389.65

[174 rows x 1 columns]

2:
          spendings_sum_abs_dif
mcc_code                       
6011               3.085862e+09
4829               2.499153e+09
6010               2.208580e+08
5541               1.554326e+08
5511               7.352877e+07
5812               7.037856e+07
5977               4.241762e+07
5533           

## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте количество транзакций с amount<0 в ночное время для мужчин и женщин. Ночное время - это примерно 00-06 часов. 

In [131]:
from datetime import datetime, timedelta

def parse_time(datetime_str):
    time = datetime_str.split()[1]
    seconds = time.split(':')[-1]
    if seconds == '60':
        datetime_str = datetime_str[:-2] + '59'
        time = datetime.strptime(datetime_str.split()[1], '%H:%M:%S')
        time += timedelta(seconds=1)
    else:
        time = datetime.strptime(datetime_str.split()[1], '%H:%M:%S')

    return time

df['tr_hour'] = df['tr_datetime'].apply(lambda datetime_str: parse_time(datetime_str).hour)
print(f"1:")
df

1:


,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,mcc_description,tr_description,tr_hour
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,10
1,39026145,6 07:08:31,4814,1030,-5614.79,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,7
2,39026145,8 07:06:10,4814,1030,-1122.96,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,7
3,39026145,11 08:49:03,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,8
4,39026145,11 14:12:08,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,14
...,...,...,...,...,...,...,...,...,...,...
999579,36837532,18 23:38:60,3000,6000,1297824.89,NaN,NaN,"Авиалинии, авиакомпании",Возврат покупки. ТУ СБ РФ,23
999580,6808459,206 19:00:10,3000,6000,124693.24,NaN,NaN,"Авиалинии, авиакомпании",Возврат покупки. ТУ СБ РФ,19
999581,83759123,90 22:40:57,3000,6000,218976.79,NaN,NaN,"Авиалинии, авиакомпании",Возврат покупки. ТУ СБ РФ,22
999582,89619970,104 23:39:35,3000,6000,408307.49,NaN,NaN,"Авиалинии, авиакомпании",Возврат покупки. ТУ СБ РФ,23


In [132]:
print(f"2: male: {len(df.loc[(df['amount'] < 0) & (df['gender'] == 1) & (df['tr_hour'] >= 0) & (df['tr_hour'] <= 6)])}")
print(f"2: female: {len(df.loc[(df['amount'] < 0) & (df['gender'] == 0) & (df['tr_hour'] >= 0) & (df['tr_hour'] <= 6)])}")

2: male: 46693
2: female: 43508
